In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, _tr
from mand.core import ProfileMonitor, SummaryMonitor, PrintMonitor
from mand.core import find, addFootnote, setDependencyManager, displayMarkdown
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb
from mand.demos.trading import MarketDataSource

db = ObjectDb()
setUpDb(db)

In [2]:
class A(Entity):

    @node
    def a(self):
        addFootnote('a was called')
        return 0
    
    @node
    def b(self):
        addFootnote('b called', '1')
        self.a()
        addFootnote('b called', '2')
        
    @node
    def c(self):
        addFootnote('c called')
        self.b()
        addFootnote('c called', '2')
        
a = A()
a.c()
n = getNode(a.c)
n.footnotes

from mand.core import displayMarkdown
displayMarkdown(n.footnoteMarkdown())

1. a was called
1. b called
  * 1
  * 2
1. c called
  * 2

In [3]:
class C(Entity):
    
    @node
    def fib(self, n):
        if n < 2:
            return n
        return self.fib(n-1) + self.fib(n-2)


In [4]:
#hangs on rerun?
c = C()
print 'done'

done


In [5]:
with SummaryMonitor():
    v = c.fib(10)
print v

with SummaryMonitor():
    v = c.fib(40)
print v

# Compute activity summary (0.00 seconds of wall clock time)

|key|value|
|-|-|
|GetValue|19
|GetValue/Calc|11

55


# Compute activity summary (0.00 seconds of wall clock time)

|key|value|
|-|-|
|GetValue|61
|GetValue/Calc|30

102334155


In [6]:
class D(Entity):
    
    @node(tweakable=True)
    def v(self): 
        return 10
        
class E(D):
    
    @node(tweakable=True)
    def v(self): 
        return super(E, self).v() + 20
    
e = E()

print 'Expect 30:', e.v()
assert e.v() == 30

with Context({e.v: 40}):
    print 'Expect 40:', e.v()
    assert e.v() == 40
        
print 'Expect 30:', e.v()
assert e.v() == 30

with Context({super(E, e).v: 40}):
    print 'Expect 60:', e.v()
    assert e.v() == 60

Expect 30: 30
Expect 40: 40
Expect 30: 30
Expect 60: 60


In [7]:
class MarketInterface1(Entity):
    
    @node(tweakable=True)
    def sourceName(self):
        return 'source1'
    
    @node
    def source(self):
        return self.getObj(_tr.MarketDataSource, '%s.%s' % (self.sourceName(), self.meta.name()))
    
    @node # (tweakable=True)
    def spot(self):
        return self.source().last()
      
_tr.add(MarketInterface1)

with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_ibm.update(last=175.61)

    s2_ibm  = MarketDataSource('user_marks.IBM')
    s2_ibm.update(last=180)
    
    ibm  = MarketInterface1('IBM').write()
    
print ibm.spot()
with Context({ibm.sourceName: 'user_marks'}):
    print ibm.spot()
print ibm.spot()

175.61
180
175.61


In [8]:
from mand.demos.depmanager import DM1

class DM2(DM1):
    
    def __init__(self):
        super(DM2, self).__init__()
    
    def simplify(self, key):
        if key.fullName() == 'MarketInterface1:spot':
            obj = key.object()
            
            dep1 = obj.sourceName
            dep2 = obj.source().clock().cutoffs
            
            return (dep1, dep2)
            
    def XXXgetNode(self, ctx, key):
        n = ctx._get(key)
        if n:
            return n
        s = self.simplify(key)
        if s: 
            nodes = [ getNode(bm) for bm in s ]
            values = [ bm() for bm in s ]
            k = [ (n.object(), n.key.fullName, v) for n, v in zip(nodes, values) ]
            cKey = tuple(k)
            
            if cKey not in self.contexts:
                tweaks = dict( [ (bm, v) for bm, v in zip(s, values)])
                ctx1 = Context(tweaks, 'simp')
                self.contexts[cKey] = ctx1
            ctx1 = self.contexts[cKey]
            ret = ctx1.get(key)
            ret.isSimplified = True
        else:
            ret = super(DM1, self).getNode(ctx, key)
            ret.isSimplified = False
        return ret
    

    
dm = DM2()
setDependencyManager(dm)

db4 = db.copy()   
ibm2 = db4.get(ibm.meta.path())

ts = Timestamp()


with Context({ibm2.sourceName: 'user_marks'}, 'user'):
    with SummaryMonitor():
        v = ibm2.spot()
    print v
    n = getNode(ibm2.spot)
    n.printInputGraph()

    displayMarkdown(n.footnoteMarkdown())
    
print ibm2.spot()


# Compute activity summary (0.03 seconds of wall clock time)

|key|value|
|-|-|
|Context|1
|Db.Get|6
|GetValue|32
|GetValue/Calc|16

180
<Node: /Global/MarketInterface1/IBM.(MarketInterface1:spot)() in Root:user:simp @4431282384 T=False>, nIn=2
   <Node: /Global/MarketDataSource/user_marks.IBM.(ExternalRefData:state)() in Root:user:simp @4431279696 T=False>, nIn=1
      <Node: /Global/MarketDataSource/user_marks.IBM.(RefData:state)() in Root:user:simp @4431281808 T=False>, nIn=4
         <Node: /Global/Clock/MarketData.(Clock:cutoffs)() in Root:user:simp @4431282320 T=True>, nIn=0
         <Node: /Global/MarketDataSource/user_marks.IBM.(MarketDataSource:clock)() in Root:user:simp @4431357392 T=False>, nIn=0
         <Node: /Global/RefDataUpdateEvent/96f648d8-e2eb-48c8-93c2-ee1fdd92b0aa.(Event:amends)() in Root:user:simp @4431357072 T=False>, nIn=0
         <Node: /Global/RefDataUpdateEvent/96f648d8-e2eb-48c8-93c2-ee1fdd92b0aa.(RefDataUpdateEvent:data)() in Root:user:simp @4431357520 T=False>, nIn=0
   <Node: /Global/MarketInterface1/IBM.(MarketInterface1:source)() in Root:user:simp @4431281360 T=False>, nIn=1
      

175.61
